### Adversarial examplesを試す前に
KerasでMNISTデータの分類をしてみる

In [4]:
from __future__ import absolute_import, division, print_function
import numpy as np
from sklearn import metrics
import tensorflow as tf
from keras.models import Model, Input
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.backend import tensorflow_backend
from keras.utils.np_utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

In [2]:
def CNN_Model(input_shape, output_dim):
    __x = Input(shape=input_shape)
    __h = Conv2D(filters=32, kernel_size=3, activation='relu')(__x)
    __h = Conv2D(filters=64, kernel_size=3, activation='relu')(__h)
    __h = MaxPooling2D(pool_size=(2, 2))(__h)
    __h = Dropout(rate=0.25)(__h)
    __h = Flatten()(__h)
    __h = Dense(units=128, activation='relu')(__h)
    __h = Dropout(rate=0.25)(__h)
    __y = Dense(units=output_dim, activation='softmax')(__h)
    return Model(__x, __y)

In [3]:
def main():
    # GPU configulations
    config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    session = tf.Session(config=config)
    tensorflow_backend.set_session(session)

    # random seeds
    np.random.seed(1)
    tf.set_random_seed(1)

    # parameters
    n_classes = 10
    n_channels = 1
    img_width = 28
    img_height = 28

    # load the dataset
    print('Loading the dataset...')
    from keras.datasets import mnist as dataset
    (X_train, Y_train_int), (X_test, Y_test_int) = dataset.load_data()
    X_train = X_train[:, np.newaxis].transpose((0, 2, 3, 1)).astype('float32') / 255.0
    X_test = X_test[:, np.newaxis].transpose((0, 2, 3, 1)).astype('float32') / 255.0
    Y_train = to_categorical(Y_train_int, num_classes=n_classes)
    Y_test = to_categorical(Y_test_int, num_classes=n_classes)

    # training
    print('Train a NN model...')
    ## define
    input_shape = (img_width, img_height, n_channels)
    model = CNN_Model(input_shape, n_classes)
    optimizer = Adam()
    model.compile(optimizer=optimizer, loss=categorical_crossentropy, metrics=['accuracy'])
    ## train
    history = model.fit(X_train, Y_train, batch_size=100,
                        epochs=1, shuffle=True, initial_epoch=0)

    # test
    Y_train_pred = model.predict(X_train)
    Y_train_pred = Y_train_pred.argmax(axis=1)
    Y_test_pred  = model.predict(X_test)
    Y_test_pred = Y_test_pred.argmax(axis=1)
    print('Training score for a NN classifier: \t{0}'.format(
        metrics.accuracy_score(Y_train_int, Y_train_pred)))
    print('Test score for a NN classifier: \t{0}'.format(
        metrics.accuracy_score(Y_test_int, Y_test_pred)))
    print('Training classification report for a NN classifier\n{0}\n'.format(
        metrics.classification_report(Y_train_int, Y_train_pred)))
    print('Test classification report for a NN classifier\n{0}\n'.format(
        metrics.classification_report(Y_test_int, Y_test_pred)))

In [4]:
main()

Loading the dataset...
Train a NN model...
Epoch 1/1
60000/60000 [==============================] - 233s 4ms/step - loss: 0.1816 - acc: 0.9458
Training score for a NN classifier: 	0.9871333333333333
Test score for a NN classifier: 	0.9842
Training classification report for a NN classifier
             precision    recall  f1-score   support

          0       0.99      0.99      0.99      5923
          1       0.99      0.99      0.99      6742
          2       0.98      0.99      0.99      5958
          3       0.99      0.99      0.99      6131
          4       0.99      0.98      0.99      5842
          5       0.98      0.99      0.98      5421
          6       0.98      0.99      0.99      5918
          7       0.98      0.99      0.99      6265
          8       0.99      0.97      0.98      5851
          9       0.98      0.98      0.98      5949

avg / total       0.99      0.99      0.99     60000


Test classification report for a NN classifier
             precision 

#### Adversarial exampleを試す

In [3]:
from cleverhans.utils_tf import model_train, model_eval
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils_keras import KerasModelWrapper
import logging
from cleverhans.utils import set_log_level

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
def CNN_Model(input_shape, output_dim):
    # core model
    __x = Input(shape=input_shape)
    __h = Conv2D(filters=32, kernel_size=3, activation='relu')(__x)
    __h = Conv2D(filters=64, kernel_size=3, activation='relu')(__h)
    __h = MaxPooling2D(pool_size=(2, 2))(__h)
#    __h = Dropout(rate=0.25)(__h)
    __h = Flatten()(__h)
    __h = Dense(units=128, activation='relu')(__h)
#    __h = Dropout(rate=0.25)(__h)
    __y = Dense(units=output_dim, activation='softmax')(__h)
    # return
    return Model(__x, __y)

In [6]:
def main():
    # GPU configulations
    config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    session = tf.Session(config=config)
    tensorflow_backend.set_session(session)

    # random seeds
    np.random.seed(1)
    tf.set_random_seed(1)

    # parameters
    n_classes = 10
    n_channels = 1
    img_width = 28
    img_height = 28

    # load the dataset
    print('Loading the dataset...')
    from keras.datasets import mnist as dataset
    (X_train, Y_train_int), (X_test, Y_test_int) = dataset.load_data()
    X_train = X_train[:, np.newaxis].transpose((0, 2, 3, 1)).astype('float32') / 255.0
    X_test = X_test[:, np.newaxis].transpose((0, 2, 3, 1)).astype('float32') / 255.0
    Y_train = to_categorical(Y_train_int, num_classes=n_classes)
    Y_test = to_categorical(Y_test_int, num_classes=n_classes)

    # training
    print('Train a NN model...')
    input_shape = (img_width, img_height, n_channels)
    model_keras = CNN_Model(input_shape, n_classes)
    
    # Kerasモデルをラッパ cleverhans.utils_keras.KerasModelWrapper でラップする
    model_cleverhans = KerasModelWrapper(model_keras)
    
    # 敵対的サンプルをどのように計算するかについては，
    # Tensorflow placeholder を用いて，その計算方法をモデルの外側で明示的に与える
    x = tf.placeholder(tf.float32, shape=(None, img_width, img_height, n_channels))
    y = tf.placeholder(tf.float32, shape=(None, n_classes))
    
    ## adversarial examples の生成に FastGradientMethod を利用する
    fgsm = FastGradientMethod(model_cleverhans, sess=session)
    fgsm_params = {'eps': 0.25, 'clip_min': 0.0, 'clip_max': 1.0}
    x_adv = fgsm.generate(x, **fgsm_params)
    
    ## 学習
    rng = np.random.RandomState([2222, 22, 2])
    train_params = {'nb_epochs': 1, 'batch_size': 100, 'learning_rate': 0.001}  # Adam param
    model_train(session, x, y, model_keras(x), X_train, Y_train, 
                predictions_adv=model_keras(x_adv), args=train_params, save=False, rng=rng)

    # test
    Y_train_pred  = model_keras.predict(X_train)
    Y_train_pred = Y_train_pred.argmax(axis=1)
    Y_test_pred  = model_keras.predict(X_test)
    Y_test_pred = Y_test_pred.argmax(axis=1)
    print('Training score for a NN classifier: \t{0}'.format(
        metrics.accuracy_score(Y_train_int, Y_train_pred)))
    print('Test score for a NN classifier: \t{0}'.format(
        metrics.accuracy_score(Y_test_int, Y_test_pred)))
    print('Training classification report for a NN classifier\n{0}\n'.format(
        metrics.classification_report(Y_train_int, Y_train_pred)))
    print('Test classification report for a NN classifier\n{0}\n'.format(
        metrics.classification_report(Y_test_int, Y_test_pred)))

In [7]:
main()

Loading the dataset...
Train a NN model...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Training score for a NN classifier: 	0.9841833333333333
Test score for a NN classifier: 	0.9799
Training classification report for a NN classifier
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5923
          1       0.98      1.00      0.99      6742
          2       0.99      0.98      0.98      5958
          3       0.99      0.97      0.98      6131
          4       0.99      0.97      0.98      5842
          5       0.97      0.99      0.98      5421
          6       0.99      0.99      0.99      5918
          7       0.98      0.99      0.99      62